<div class="alert alert-block alert-info" style="color:black"> <h2>Activity 5: Investigate the time and space (memory) requirements of your two methods</h2>
    You should now have working versions of both breadth-first and (restricted) depth-first search. They already store the number of attempts tested (a measure of runtime), and the code cells that run them print that value out.<br>
    The next step is to compare memory - which is proportional to the maximum size of the open list. 
    <br><br><b>How to get started:</b> Edit your code for both classes:
    <ol>
    <li> Copy-paste <code>update_working_memory()</code> into your <code>BreadthFirstSearch</code> class</li>
    <li> In both your classes add a new parameter <code>self.max_memory</code> with a default value 0 by over-riding the <code>__init__()</code> method of the super class.</ul>
    <li> Override <code>update_working_memory()</code> in both your classes, adding code to:
        <ul>
            <li>check the length of the open_list against <code>self.max_memory</code></li>
            <li> update the value of <code>self.max_memory</code> if the open list has increased in size.</li>
        </ul>
    <li> Copy-paste the testing code from the cells above, then adapt it to test the time and memory needs of your algorithms.</li> 
    </ol>
    <b>Note:</b> this is a <em>Stretch</em> activity so don't worry if you can't complete it easily.
</div>

In [1]:
# YOU MUST RUN THIS CELL BUT DO NOT EDIT IT OR YOU WILL BREAK THE NOTEBOOK
import sys, os

# Import from the common directory
sys.path.append('../common')
from problem import Problem
from candidatesolution import CandidateSolution
from singlemembersearch import SingleMemberSearch
from combinationproblem import CombinationProblem

In [2]:
class BreadthFirstSearch(SingleMemberSearch):
    def __init__(
        self,
        problem: Problem,
        constructive: bool = False,
        max_attempts: int = 50,
        minimise=True,
        target_quality=1,
    ):
        super().__init__(problem, constructive, max_attempts, minimise, target_quality)
        self.max_memory = 0  

    def __str__(self):
        return "breadth-first"

    def select_and_move_from_openlist(self) -> CandidateSolution:
        next_soln = self.open_list[0]
        self.open_list.pop(0)
        return next_soln

    def update_working_memory(self, neighbour: CandidateSolution, reason: str):
        if neighbour.quality == self.target_quality:
            self.result = neighbour.variable_values
            self.solved = True
        elif reason != "":
            self.runlog += (
                f"discarding invalid solution {neighbour.variable_values} because {reason}\n"
            )
            self.closed_list.append(neighbour)
        else:
            self.runlog += (
                f"adding solution to openlist: {neighbour.variable_values}\tquality {neighbour.quality}\n"
            )
            self.open_list.append(neighbour)
            if len(self.open_list) > self.max_memory:
                self.max_memory = len(self.open_list)


In [3]:
class RestrictedDepthFirstSearch(SingleMemberSearch):
    def __init__(
        self,
        problem: Problem,
        constructive: bool = False,
        max_attempts: int = 50,
        minimise=True,
        target_quality=1,
        max_depth: int = 10,
    ):
        super().__init__(problem, constructive, max_attempts, minimise, target_quality)
        self.max_depth = max_depth
        self.max_memory = 0  

    def __str__(self):
        return "restricted-depth-first"

    def select_and_move_from_openlist(self) -> CandidateSolution:
        next_soln = self.open_list[-1]
        self.open_list.pop()
        return next_soln

    def update_working_memory(self, neighbour: CandidateSolution, reason: str):
        if neighbour.quality == self.target_quality:
            self.result = neighbour.variable_values
            self.solved = True
        elif reason != "":
            self.runlog += (
                f"discarding invalid solution {neighbour.variable_values} because {reason}\n"
            )
            self.closed_list.append(neighbour)
        elif len(neighbour.variable_values) < self.max_depth:
            self.runlog += (
                f"adding solution to openlist: {neighbour.variable_values}\tquality {neighbour.quality}\n"
            )
            self.open_list.append(neighbour)
            if len(self.open_list) > self.max_memory:
                self.max_memory = len(self.open_list)


In [4]:
def test_search_memory():
    goal = [9, 9, 9, 9]

    problem = CombinationProblem(tumblers=4, num_options=10)
    problem.set_goal(goal)

    bfs = BreadthFirstSearch(problem, max_attempts=10000)
    bfs.run_search()
    print(f"BFS = Trials: {bfs.trials}, Max Memory: {bfs.max_memory}")

    dfs = RestrictedDepthFirstSearch(problem, max_attempts=10000, max_depth=5)
    found = dfs.run_search()
    print(f"DFS (max_depth=5) = Found: {found}, Trials: {dfs.trials}, Max Memory: {dfs.max_memory}")

test_search_memory()


BFS = Trials: 9180, Max Memory: 6821
DFS (max_depth=5) = Found: True, Trials: 3831, Max Memory: 3607


In [5]:
import workbook2_utils as wb2
display(wb2.q12)
display(wb2.q13)